# DA201: Data Analytics using Python
# Main Assignment

## Introduction

I will be using this Jupyter Notebook to produce the data analysis for the assignment. Insights from the analysis will be recorded on this notebook too alongside the analytical code snippet. A version of this notebook along with all associated datafiles have been uploaded into a special repo within my GitHub Page. It is linked further down below.

## Preparing the environment

In [ ]:
# Import libraries required.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import bs4
from bs4 import BeautifulSoup
import string
from datetime import datetime
from matplotlib import rcParams
from sklearn import linear_model


# Set plotting options.
sns.set_palette("pastel")
sns.set(font_scale=2)

# Setting up date-parser using a lambda.
d_parser = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')

# Import the CSV files containing the data required for the Assignment.
# Running the date-parser
tweets = pd.read_csv('tweets.csv')
cov = pd.read_csv('covid_19_uk_cases.csv', parse_dates=['Date'], date_parser=d_parser)
vac = pd.read_csv('covid_19_uk_vaccinated.csv', parse_dates=['Date'], date_parser=d_parser)

## Hyperlinking URL to My GitHub Repo

- All work relating to this Assignment can be found on my [GitHub Pages](https://github.com/SauravChakers?tab=repositories).

- Screenshot demo. 

!['My Github screenshot](https://github.com/SauravChakers/Data_Analytics_using_Python/blob/main/MyGitHub%20Screenshot.png?raw=True)



## What is GitHub?
- A free online resource for organising, storing and sharing code and related work with others.
- Cloud storage for any file type but is most often used for code files and related supporting documents.
- A platform to collaborate, network and pitch work amongst developers in real time.
- The most widely used version control system amongst developers and coders.

## How does GitHub add value?
- It has a built-in version control feature which can then be used to:
    - Track changes and those responsible for the changes. 
    - Create localised copy away from production version where upgrades or fixes can first be tested ahead of implementation.
    - Roll back to previous versions. 
- Version control is useful on collaborative projects where simultaneous work may be required.
- It can be used to track progressive performance of coders.
- As it is open source, projects can potentially draw upon resources of the entire community of coders.
- Provide a simple back up of work in the cloud asides from any local networks and hardware.

## Data Exploration

### Tweets DataFrame

In [ ]:
# Determine the number of rows and columns.
print(tweets.shape)

# Can see 3960 rows and 21 columns.
# Implies we have 3960 tweets in the dataframe.

In [ ]:
# Determine the data types in the DataFrames.
print(tweets.dtypes)

# Can see this is NOT all text data.
# Should be able to rank the most favourite or retweeted content

In [ ]:
# Get information on the contents of the columns.
tweets.info()

# Looks like the main content of each tweet is contained in the column 'text'.

In [ ]:
# View the first 5 rows to see whats in each column.
tweets.head()

# Confirmed that 'text' column contains the main content of the tweets.
# Contains # 

In [ ]:
# View the last 5 rows.
print(tweets.tail())

# Can see from heads and tails that there are 21 columns and no footer.

In [ ]:
# Determine null values.
tweets.isnull().sum()

# Can see 1 row may have all columns blanks.
# Can see at 830 tweets were considered as possibly sensitive by Twitter algos.

### Cases DataFrame 

In [ ]:
# Determine key characteristics of each DataFrame.
# View the first 5 rows.
print(cov.head())

# Can see index value 0 is header.
# Date Values are in rows i/o columns.
# Time-series often easier if time values are on columns.
# Lots of geographical data; need to consider usefulness.

In [ ]:
# View the last 5 rows.
print(cov.tail())

# All rows until the end have changes in row values.

In [ ]:
# Determine the number of rows and columns.
print(cov.shape)

# Can see 7584 rows and 21 columns.
# Date format is dd/mm/yyyy
# Changing to date parser at load -in.

In [ ]:
# Determine the data types in the DataFrames.
print(cov.dtypes)
# Can see date parser at load-in worked.
# Date in correct pandas format.

In [ ]:
# Determine null values.
cov.isnull().sum()

# Can see very few missing values.
# Will determine what to do with them later.

### Vaccinated DataFrame

In [ ]:
# Determine key characteristics of each DataFrame.
# View the first 5 rows.
print(vac.head())

# Can see index value 0 is header.

In [ ]:
# View the last 5 rows.
print(vac.tail())

# Perhaps the last rows dont have any data; need to check.
# Will change dates to columns i/o rows here too.

In [ ]:
# Determine the number of rows and columns.
print(vac.shape)

# Can see 7584 rows and 11 columns.

In [ ]:
# Determine the data types in the DataFrames.
print(vac.dtypes)

# Can see date parsing successful here too.

In [ ]:
# Remove spaces in columns name
vac.columns = vac.columns.str.replace(' ','_')

# Makes running formulas easier.

In [ ]:
# Determine null values.
vac.isnull().sum()

# Can see no missing values.
# Given all column names are shared with Cases it may be a good idea to merge the two DataFrames.
# Do we need all this geographical data? Removing it might make the DataFrame more resource efficient.

## Filtering the data for region Gibraltar

I will filter the data for a small subset.
This should give me an idea of how to approach the wider data set once it has been wrangled.

### Cases Data

In [ ]:
# Are there default column names?
list(cov.columns)

# Yes, headers are defined as default column names.

In [ ]:
# Filter the data by Province/State = Gibraltar
# Gib is a boolean variable with True or False in it.
Gib = cov[cov['Province/State'] == 'Gibraltar']

In [ ]:
# Calculating daily changes to Deaths, Cases, Recovered.
# Create new Column for each variable with 'Day' prefix.
# Calculate the daily change using shift(1) and subtract functions.
Gib['Day_Deaths']=Gib['Deaths'] -Gib['Deaths'].shift(1)
Gib['Day_Cases']=Gib['Cases'] -Gib['Cases'].shift(1)
Gib['Day_Recovered']=Gib['Recovered'] -Gib['Recovered'].shift(1)

# There may be a neater way to do this calculation using loc function.

In [ ]:
# Checking when Gibraltar first reported a case.
# This should should 1 in Daily_Cases column.
# Need to filter in column Cases > 0
Filt_Gib = Gib[Gib['Cases'] > 0] 
Filt_Gib



# Proof that the daily changes calculations above work!
# First case reported on 4 Mar 2020.
# Total 97 Deaths in Gibraltar for the period of the data.
# Total 5727 Cases in Gibraltar for the period of the data.
# See that Daily Cases at start of period of data was very low.
# See that Daily Cases at end of period of data still comparatively high.
# Note multiple peaks and troughs in daily data through the period of the data.

In [ ]:
# When was first death recorded in Gibraltar?
# Need to filter in column Deaths > 0
Filt_Gib = Gib[Gib['Deaths'] > 0] 
Filt_Gib

# See that first Death reported in Gibraltar on 11 Nov 2020
# Daily cases at this time of c. mid 20s not much different to daily cases at end of period of data.

In [ ]:
# Aggregate Data by month.
Gib_monthly = Gib.set_index('Date', inplace=True)
Gib.index = pd.to_datetime(Gib.index)
Gib.resample('1M').sum()

# This is much easier to extract insights out of.
# Deaths in winter 2020/21 starting from Oct 2020 & peaking in Jan 2021 with 70 deaths in that month.
# Dramartic decline in Deaths after Feb 2021. Be interesting to investigate link with vaccine dataframe.
# Sharp rise in cases from Sept 2020 after which it becomes even more dramatic in the winter of 2020/21.
# Cases remain very high until Sept 2021 and but declines significantly in Oct 2021.
# Sharp rise in daily cases during Dec 2020 & Jan 2021. 
# Stabilises thereafter barring a spike in July & Aug 2021.
# Looks like strong correlation between peaks of cases, deaths and hospitalisation numbers albeit with some lags.
# Note that aggregating by month changes the deaths, cases, recovered and hospitalised.
# Will need to account for these changed ahead of analysis in the wider data.

### Vaccinated Data

In [ ]:
# Filter the data by Province/State = Gibraltar
# Gib is a boolean variable with True or False in it.
Gib = vac[vac['Province/State'] == 'Gibraltar']
Gib

# First_Dose, Second_Dose=Vaccinated are daily numbers.

In [ ]:
# Calculating difference between 'First Dose' and 'Second Dose' values.
# This will represent the individuals who have taken First Dose but not Second Dose.
# Call this 'Part_Vac'.
Gib['Part_Vac']=Gib['First_Dose'] - Gib['Second_Dose']


In [ ]:
# Checking when Gibraltar first administered the first dose.
# This should should at least 1 in First_Dose column.
# Need to filter in column First_Dose > 0
Filt_Gib = Gib[Gib['First_Dose'] > 0] 
Filt_Gib

# Can see first day of vaccine being rolled out was 11 Jan 2021.
# Strange that there were 2596 second doses also administered on that day. 
# So where are the first doses for these?
# Too many data points to visually discern any meaningful insights.
# Need to aggregate data in monthly intervals.

In [ ]:
# Aggregate Data by month.
Gib_monthly = Gib.set_index('Date', inplace=True)
Gib.index = pd.to_datetime(Gib.index)
Gib.resample('1M').sum()

# Large increase in the number of first doses being adminstered in the first few months.
# Stabilises from Mar to Jun 2021 and then declines sharply.
# One month lag between peaks of first & second dose.
# Part vaccinated declines sharply as second dose adminstrations start to outstrip first dose.
# No meaningful insight aside from above on part vaccinated.
# Time Series & correlations might make it easier to understand.
# Need to check aggregates.

In [ ]:
# Trying pairplot to visualise relationships between the variables.
sns.pairplot(Gib)

# Too many columns.
# Very noisy visualisation of Dataframe.
# Will return on the wider data set once it has been wrangled.

## Expectations on the wider data

Filtering the Data so far for just Gibraltar was an useful exercise. For the fuller data, this exercise has shown:
- 1. Where calculated columns need to be added & manipulated to incorporate changes made during aggregation.
- 2. Some columns which can be deleted as they contain no meaningful data for our analysis.
- 3. Daily data is too noisy to gather any insights.
- 4. Data will need to be resampled into monthly output.
- 5. The two datasets Cov & Vac should be merged.
- 6. Total numbers are useful to highlight local peaks and troughs as well factors such as seasonality.
- 7. Trends over time offer broader insights into interdependence between variable.
- 8. Full analysis will need to consider both totals as well as timeseries.
- 9. Visualisation not useful at this stage as there is too much noise in the data: too many unncesesary columns, very large variation in data scales and very likely the presence of outliers.
- 10. We now know how to wrangle the data best such that we may anlyse it.

## Data Wrangling

### Merging Cases & Vaccinated DataFrames 

In [ ]:
#'covid' DataFrame will merge 'cov' and 'vac' DataFrames.
covid = pd.merge(cov, vac, left_index=True, right_index=True)

print(covid.shape)
print(cov.shape)
print(vac.shape)

# Can see that there are the same number of rows in covid as there were in cov and vac.
# Can see that the number of columns in covid add up to the total columns in cov and vac.

### Cleaning up the Merged DataFrame

#### Shape of the Merged DataFrame

In [ ]:
# Lets first pull up the column names in 'covid' to identify duplications & redundancies.
# Will also determine data types at the same time.
print(covid.dtypes)

# Delete all duplicate and redundant columns from merged DataFrame 'covid'.

In [ ]:
# Checking for any duplicated columns & dropping them.
covid = covid.loc[:,~covid.apply(lambda x: x.duplicated(),axis=1).all()].copy()
print(covid.dtypes)

# All columns which are duplicated are deleted/dropped.

In [ ]:
# Delete redundant columns from the dataframe 'covid'.
# These columns contain no useful information for our analysis.
covid_cleaned = covid.drop(covid.columns[[1, 2, 3, 4, 5, 6]], axis=1) 
# Realigning names of some columns in covid_cleaned to original dataset.
covid_cleaned.rename(columns={"Province/State_x": "Province/State", 
                              "Date_x": "Date"}, inplace=True)

In [ ]:
# Check data types & shape of covid
print(covid_cleaned.dtypes)
print(covid_cleaned.shape)

# Output as designed.

#### Dealing with Missing values

In [ ]:
# Check for missing/null values.
covid_cleaned.isnull().sum()

# Can see 2 rows have nulls values
# Ties up with earlier wrangling done on Cases DataFrame.
# Selective 0 values should be replaced.

In [ ]:
# Filter out rows which does not contain any values in the columns of interest.
# This reduces demand for computational power.
covid_clean = covid_cleaned.drop(covid_cleaned[(covid_cleaned['Deaths'] == 0) & 
                            (covid_cleaned['Cases'] == 0) & 
                            (covid_cleaned['Recovered'] == 0) & 
                            (covid_cleaned['Hospitalised'] == 0) &
                            (covid_cleaned['Vaccinated'] == 0) & 
                            (covid_cleaned['First_Dose'] == 0)].index)

# covid_clean is the new baseline DataFrame to use.
# Will use this for all analysis going forward.
print(covid_clean.shape)
# Reduces rows from 7584 to 6942.
# Will enhance efficiency of DataFrame.

In [ ]:
# View the 2 rows with the null values.
covid_clean[covid_clean.isna().any(axis=1)]


In [ ]:
# Lets examine the DateFrame around these rows.
# Will help determine how these null values should be replaced.
covid_clean.iloc[750:758]

# No changes to any column except Hospitalised on these two days.
# We can backfill Hospitalised for the missing values given values are rising.
# We can replace other NaN values either with backfill or forward fill.
# As there is no change in the values either direction.

In [ ]:
# Replace missing values with backward fill in Hospitalised. 
covid_clean['Hospitalised'] = covid_clean['Hospitalised'].fillna(method='bfill')
# Replace other Null values
covid_clean = covid_clean.fillna(method='ffill')
#Check for any other null values
covid_clean[covid_clean.isna().any(axis=1)]

# No more null values left.

In [ ]:
# Recheck the same rows to verify what values have been inserted.
covid_clean.iloc[750:758]

# Replacements successful.

In [ ]:
# Check for duplicate rows in the DataFrame.
duplicateRows = covid_clean[covid_clean.duplicated()]

#view duplicate rows
duplicateRows

# No Duplicates found. 

### Data Analysis

#### Adding Calculated Columns

In [ ]:
# We need to identify the difference between application of First & Dose.
# This represents the individuals who have taken the First Dose of the vaccine but not Second Dose.
# This is the target group for the marketing campaign.
# Call this 'Part_Vac'.
covid_clean['Part_Vac'] = covid_cleaned['First_Dose'] - covid_cleaned['Vaccinated']

# Top row in all calculated columns will return Null value as there is no preceeding value.
# This is expected when using shift function for calculations which requires a preceedig value to perform the calculation.
# Replace this row NaN values with 0.
covid_clean.fillna(0)

In [ ]:
# Upsample datetime objects into month and year.
# Will be needed for aggregate functions.
# Will reduce the frequency of the data.
# Create new columns Month & Year.

covid_clean['Month'] = covid_clean['Date'].dt.month
covid_clean['Year'] = covid_clean['Date'].dt.year
covid_clean['Week'] = covid_clean['Date'].dt.week
covid_clean.info()

#### Grouping Data

In [ ]:
# Changing the index to Date.
# This converts the DataFrame into a timeseries data.
covid_clean.set_index('Date', inplace=True)
covid_clean.info()

# With Date as Index, we can now filter DataFrame with Dates.
# Much easier to work with for time series data in this format.
# No need to transpose dates into column values.
# Avoids creating a very wide DataFrame which is potentially more unwieldy.

In [ ]:
# Basic visualisation to check if time series created.
covid_clean.Hospitalised.plot(title="Hospitalisations over time")
plt.tight_layout(); plt.show

# Shows that creation of time series is successful.

In [ ]:
# Identifying the highest number of individuals who have recieved first dose but not second dose.
# This is a point in time static identification of this value.
# This is available in Part_Vac Column
# Identifying max value in Part Vac_Column & returning all corresponding row values to identify Province/State

covid_clean[covid_clean['Part_Vac']==covid_clean['Part_Vac'].max()]

# So Gibraltar had the highest number of individuals who had received the first dose but not second dose
# This was on 20 Mar 2021.

In [ ]:
# Looking for peak roll out of first dose
# This period represents the peak month for the administration of the first dose.
covid_clean.groupby('Month')['Part_Vac'].sum()

# We can see that this was Month 2 = Feb 2021
# On a monthly aggregate, the highest number of individuals who received first dose but not second dose was in 02/21.
# The peak value for such people = Part_Vac was 10,657,478

In [ ]:
# Show how Part_Vac evolved over time in each 'Province/State'.
# Group the DataFrame by multiple Groups as columns.

covid_clean.groupby(['Month','Province/State'], as_index=False)['Part_Vac'].sum()

In [ ]:
# Group the DataFrame for multiple functions.
# Region wide month on month break down of total and average first & second dose administered.
# Recall that Second dose = Vaccinated.
covid_clean.groupby(['Month','Year','Province/State'], as_index=False)['First_Dose', 'Vaccinated'].agg(['sum', 'mean'])

In [ ]:
# Identifying highest % of individuals who have received first dose but not second dose.
# Will need to add calculated column.
# Not clear what the denominator in this function should be
# Individuals who received first dose but not second dose = Part_Vac.
# The denominator is individuals who have recieved just First Dose.
# First Dose is a proxy for eligibility for  receiving Second Dose too. 
# This is a good indicator of "riskiness" of any row of data.
# In theory this is where vaccination campaign needs to target as least people are fully vaccinated vs eligibility

# Inserting Calculated Column
covid_clean['Ratio_of_Int'] = ((covid_clean['Part_Vac'])*100 / covid_clean['First_Dose'])

# The lower the value in this column, the worse it is!

# Identifying max value in % at Risk column & returning all corresponding row values to identify Province/State
covid_clean[covid_clean['Ratio_of_Int']==covid_clean['Ratio_of_Int'].min()]

# This is in the Province/State = '	Saint Helena, Ascension and Tristan da Cunha'
# This is a point in time observation.

In [ ]:
# Show monthly aggregated for Ratio_of_Int.
covid_clean.groupby('Month')['Ratio_of_Int'].sum()

# Confirms again that Month 2 = Feb 2021 was peak month when most people had received just the first dose.
# In this example, this feature is called Peak Ratio of Interest.

In [ ]:
# Calculating Ratio of people who have received seond dose over the first dose.
# Calling this 'First Percentage'
# Inserting Calculated Column
covid_clean['First_Percentage'] = (covid_clean['First_Dose']) / (covid_clean['Vaccinated'])

# See what the DataFrame now looks like after addition of all calculated columns.
covid_clean

In [ ]:
# Rearrange columns so that they have a more logical order in the DataFrame.
# Makes the DataFrame look neater too.

covid_clean = covid_clean[['Week', 'Month', 'Year', 'Province/State', 'Deaths', 'Cases', 'Recovered', 
                           'Hospitalised', 'First_Dose', 'Part_Vac', 'Vaccinated', 
                           'First_Percentage', 'Ratio_of_Int']]

# Check how the rearranged DataFrame looks.
covid_clean

# Confirmed that the columns now flow more logically and looks neater.

In [ ]:
# To see if there is are differences over time in application of First and Second Dose.
# The difference between application of First & Second Dose is available in Part_Vac Column
# Lets pull up how the difference between application of First and Second Dose has evolved over monthly intervals.
# Resample the data for Monthly aggregate of this column.
t_series_Part_Vac=covid_clean['Part_Vac'].resample('M').sum()
t_series_Part_Vac

# We can see that Vaccine Roll out was in  Jan 2021
# We can see that until Mar 2021, there were many more First Doses administered vs Second Dose.
# From April 2021, we can see that the trend reverses.
# More Second Doses being adminstered than First Dose.
# This is the period when the vaccinated numbers will rise.
# Quickly resample the dataframe to confirm below.
# Interesting to note that again in Oct 2021, there is a period when first doses > second dose.
# Likely that those who didnt take the vaccine in the first wave are now complying with vaccine mandate.

In [ ]:
# Creating a basic visualisation
%matplotlib inline
t_series_Part_Vac.plot()
plt.tight_layout()

# Visualisation is basic.

In [ ]:
# Resampling the entire dataframe into Monthly aggregates.
# This is for across all the regions in the UK.
cov_agg = covid_clean.resample('M').agg({'Cases': 'sum', 'Deaths': 'sum', 'First_Dose': 'sum', 
                                         'Vaccinated': 'sum', 'Part_Vac': 'sum', 'Ratio_of_Int': 'sum',
                                        'First_Percentage': 'sum'})
# Lets see the DataFrame
cov_agg

# We are missing regional variations but good overview of how each variable evoloved on a monthly basis.
# So we can see that end of 2020, there were 2,488,780 cases of Covid-19 reported across the UK.
# By the end of 2020, over 73,512 people had died from Covid-19 infection across the UK.
# Death rates continue to rise sharply until Mar 2021.
# This coincides with above where we saw that the bulk of the population was partly vaccinated by then.
# Suggests that even just the first dose of the vaccines may have helped to reduce deaths.
# At the very least, they helped reduce the rate of increase of deaths.

In [ ]:
# A basic visualisation of the key columns will help really emphasise findings above. 
# Columns to be visualised are "Deaths", "Cases", "First_Dose"
# Big differences in scale of range of data so will plot on log
# Set out the lineplot variables.
g=sns.lineplot(x="Date", y="Deaths", data=cov_agg)
g=sns.lineplot(x="Date", y ="Cases", data=cov_agg)
g=sns.lineplot(x="Date", y ="First_Dose", data=cov_agg)

# Set y axis scale.
g.set_yscale('log')

# Set out chart title.
g.set_title('TimeSeries on Key Aggregate Variables')

# Set out legend.
g.legend(['Deaths', 'Cases', 'First_Dose'])

# Confirms that Deaths flatline once the vaccine is introduced.
# Need to do better on scales etc on these visualisations.

### Finding and cleaning for Outliers using z-score

In [ ]:
# Before handling outliers, I will identify them for each column.
# Boxplot is best for visualising outliers.
rcParams['figure.figsize'] = 40,20
sns.boxplot(data=covid_clean).set(title='Chart1')
sns.despine(offset=10, trim=True);

# Column Values are on too many orders of magnitude to visualise in one chart.
# For visualisation, Use log scales to negate the differences in scales of the column values.

In [ ]:
# Using log scale to replot.
g=sns.boxplot(data=covid_clean)
g.set_yscale('log')
g.set_title('Boxplot of Key Variables on log scale')
plt.show()

# Visualisation much clearer.
# Lots of values outside upper fence.
# Values outside upper fence = 'outliers'.

In [ ]:
# Checking for distribution tails to measure "quality of outliers".
# Violinplot is best for visualising distribution tails. 
# Multi-modal distribution tails are not outliers.
# Remove unnecessary and dependant variables to improve the visualisation's focus.
covid_clean2 = covid_clean.drop(['Week', 'Month', 'Year', 'Province/State', 
                                 'Part_Vac', 'First_Percentage', 'Ratio_of_Int'], axis=1)

# Normalise the modified dataframe to narrow down the range of the magnitude of the variables
# Doesnt alter statistical properties of the variables.
normalized_covid_clean2=(covid_clean2-covid_clean2.mean())/covid_clean2.std()

# Set out the plot.
g=sns.violinplot(data=normalized_covid_clean2, showmedians=True, showmeans=True)
g.set_title('Chart2')

# Show plot
plt.show()

# Can see that all columns are fat tailed although Hospitalised less so.
# Some evidence of multi-modality in Deaths, Cases & Recovered.
# A column by column differentiate approach to outlier cleaning warranted.
# This will maintain distribution qualities such as multimodality.
# Beyond this, outliers can be replaced with interpolated values based on values from neighbouring rows.
# Outlier cleaning warranted across all columns albeit by different levels of aggresiveness.
# Deaths and Cases need most aggressive treatment as they have the most data points outside the box & most fat tailed.
# Hospitalised needs least agressive cleaning as it is comparitively thin tailed.

In [ ]:
# Using Z-Score Method to replace outliers.
# Outliers replaced with neighbouring row interpolation.
d = covid_clean.Deaths
m = ((d - d.mean()) / d.std()).abs() > 1.5
covid_clean['Deaths'] = covid_clean['Deaths'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Cases column.
c = covid_clean.Cases
m = ((c - c.mean()) / c.std()).abs() > 1.5
covid_clean['Cases'] = covid_clean['Cases'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Recovered column.
r = covid_clean.Recovered
m = ((r - r.mean()) / r.std()).abs() > 1.5
covid_clean['Recovered'] = covid_clean['Recovered'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Hospitalised column.
h = covid_clean.Hospitalised
m = ((h - h.mean()) / h.std()).abs() > 1.5
covid_clean['Hospitalised'] = covid_clean['Hospitalised'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Vaccinated column.
v = covid_clean.Vaccinated
m = ((v - v.mean()) / v.std()).abs() > 1.5
covid_clean['Vaccinated'] = covid_clean['Vaccinated'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the First_Dose column.
fd = covid_clean.First_Dose
m = ((fd - fd.mean()) / fd.std()).abs() > 1.5
covid_clean['First_Dose'] = covid_clean['First_Dose'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Part_Vac column.
pv = covid_clean.Part_Vac
m = ((pv - pv.mean()) / pv.std()).abs() > 1.5
covid_clean['Part_Vac'] = covid_clean['Part_Vac'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the Ratio_of_Int column.
roi = covid_clean.Ratio_of_Int
m = ((roi - roi.mean()) / roi.std()).abs() > 1.5
covid_clean['Ratio_of_Int'] = covid_clean['Ratio_of_Int'].mask(m).interpolate()

In [ ]:
# Apply the outlier replacement to the First_Percentage column.
fp = covid_clean.First_Percentage
m = ((fp - fp.mean()) / fp.std()).abs() > 1.5
covid_clean['First_Percentage'] = covid_clean['First_Percentage'].mask(m).interpolate()

In [ ]:
# Visualise after outliers have been replaced.
# Using boxplot first.
g=sns.boxplot(data=covid_clean)
g.set_yscale('log')
g.set_title('Boxplot of cleaned key variables on logscale ')
plt.show()

# As expected far fewer outliers across the board when compared to Chart1 above.
# Can see that the variables/columns have data on a very wide magnitude of range from the log scale below.
# Need to change the data such that all y axis values fall within a narrower range.
# This is called re-scaling/normalisation.
# For now will not normalise but instead use log scales for visualisation.
# Will revisit normalisation later if warranted.

In [ ]:
# Visualise distributions after outliers have been replaced.
# Using violinplot again.
# Remove unnecessary date parsing and calculated columns to improve the visualisation's focus.
covid_clean2 = covid_clean.drop(['Week', 'Month', 'Year', 'Province/State', 
                                 'Part_Vac', 'First_Percentage', 'Ratio_of_Int'], axis=1)

# Normalise the modified dataframe to narrow down the range of the magnitude of the variables
# Doesnt alter statistical properties of the variables.
normalized_covid_clean2=(covid_clean2-covid_clean2.mean())/covid_clean2.std()

# Setting up the plot.
g=sns.violinplot(data=normalized_covid_clean2, showmedians=True, showmeans=True)
g.set_title('Violinplot of cleaned key variables')

# Show plot
plt.show()

# Unfortunately this vis isnt great to much sense of the transformation versus earlier in Chart 2.
# This is because some time series related columns such as Week, Month etc are corrupting the visualisation.
# Clean up some unnecessary coloumns and plot again.

### Visualise & Identify Initial Trends

In [ ]:
# Plotting vertical Bar Graph
# Plotting First Percentage as a time series.
# Setting the style
sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
f, g = plt.subplots(figsize=(15, 5))

# Grouped by Province/State
df=covid_clean.groupby(['Month','Province/State'], as_index=False)['First_Percentage'].sum()


g=sns.barplot(x = covid_clean["Month"], y = covid_clean["First_Percentage"], data= df)

# Add a legend and informative axis label
g.set(xlim=(0, 24), ylabel="",
       xlabel="Sum of First Percentage by Month in 2021")
sns.despine(left=True, bottom=True)
# Setting size of display
rcParams['figure.figsize'] = 25,15

# Display
plt.show()

# Saving the Seaborn Figure:
plt.savefig('t-series_First_Percentage.png')

# Shows that there is a slow rise in Number of individuals who are eligible for the second dose who havent taken it yet.

In [ ]:
# Group the data by Province/State and Date, and aggregate the death count.
# Creating the df first per above.

df=covid_clean.groupby(['Month','Province/State'], as_index=False)['Deaths'].sum()

# Setting the size
sns.set(rc = {'figure.figsize':(15,8)})

# Setting the style
sns.set_theme(style="whitegrid")

# Visualising trend of deaths across all regions
g=sns.lineplot(x="Month", y="Deaths", data=df, hue='Province/State')
g.set_title('Deaths across regions over time')

# Very clear that death count from "Others" is skewing the data.
# Plot again on log scale.

In [ ]:
# Setting on log scale to remove the effect of vast difference in scale of numbers within the variable, Deaths. 
# Visualising trend of deaths across all regions

g=sns.lineplot(x="Month", y="Deaths", data=df, hue='Province/State')
g.set_title('Deaths across regions over time')
g.set_yscale('log')

In [ ]:
# Alternatively drop "Others" from the dataframe
# Creating a new dataframe where any rows where the Province/State = 'Others' is dropped.
# Selecting all "Province/State" except 'Others'.
df2= df[df["Province/State"].str.contains("Others") == False]

g=sns.lineplot(x="Month", y="Deaths", data=df2, hue='Province/State')
g.set_title('Deaths across regions over time excl. Others')

# Exporting the plot into a shareable PNG
plt.savefig('Deaths_across_regions_ex_Others.png')

# See that most other regions had very low deaths too as they still dont show properly.
# Probably too many variables to plot in one chart. 
# Could be neat to split this into two charts.
# Deaths have peaked and monthly aggregates in even the high death regions are still declining rapidly.

In [ ]:
# Group the data by Province/State, 
# Aggregate the count of recovered cases and sort the values of recovered cases in ascending order.
# Creating the dataframe as above
df=covid_clean.groupby(['Month','Province/State'], as_index=False)['Recovered'].count()

# Visualising the output of df on a lineplot.
g=sns.lineplot(x="Month", y="Recovered", data=df, hue='Province/State')
g.set_title('Trend of Recovered Cases over the months')

# Exporting the plot into a shareable PNG
plt.savefig('Trend_Recovered_Cases_over_months.png')

# Trends broadly the same across all Province/States.
# Big increase in Recovered Cases after Feb 2021
# Stable for the next few months.
# A big drop in recovered cases since Sep 2021
# Same time as slow rise in number of individuals who are eligible for the second dose who havent taken it yet. 
# Should look at what happens to cases reported over this period.

In [ ]:
# Plotting related aggregated data to investigate for visual relationships between some variables
# Explore impact of both doses being administered = Vaccinated on Total Deaths 
# Looks at aggregated numbers of Deaths, & administration of both first and second doses.
# Use weekly data to get more granular view

# Creating the dataframe
df=covid_clean.groupby(['Week'], as_index=False)['Deaths', 'Vaccinated'].agg({'Deaths': 'sum',  
                                                                               'Vaccinated': 'sum'})
ax = df.plot(x="Week", y="Deaths", legend=False)
ax2 = ax.twinx()
df.plot(x="Week", y="Vaccinated", ax=ax2, legend=False, color="r")
ax.set_title('Chart3')
ax.figure.legend()
plt.show()

# The variables Deaths & Vaccinated move together
# Looks like there is a lagged impact which isnt coming across here due to X axis not being fully synchronised.

In [ ]:
# There are too many columns and rows for visual checks for correlations to clearly come through.
# So many data points dont allow for particular relationships between variables being easily identified.
# We can employ Python's statistical capabilities to get a quick over-view by running correlations.
# The “corr()” method evaluates the correlation between all the variable, then it can be graphed with a color coding.

# Prepare the dataframe
# Drop  time parsing related & calculated variable columns which currently appear as independent variables in the dataframe.
# This will remove obvious but spurious correlations obscuring the analysis. 
covid_clean2 = covid_clean.drop(['Week', 'Month', 'Year', 'Province/State', 
                                 'Part_Vac', 'First_Percentage', 'Ratio_of_Int' ], axis=1)

# Setting up the correlation matrix
corr = covid_clean2.corr()

#Plotting the matrix.
fig = plt.figure()
ax = fig.add_subplot(111)

# Introducing heatmap
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)

# Setting up the display properties of the plot
ticks = np.arange(0,len(covid_clean2.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(covid_clean2.columns)
ax.set_yticklabels(covid_clean2.columns)

# Visualise the correlation matrix.
plt.show()

# When there is no correlation between 2 variables (when correlation is 0 or near 0) the color is gray. 
# The darkest red means there is a perfect positive correlation.
# Blue implies negative correlation.
# The darkest blue means there is a perfect negative correlation.
# The standout and rather obvious observation from this analysis is the positive correlation between Cases & Deaths.
# There is also reasonably strong correlation between Vaccinated & First Dose.
# This implies that those who have only taken the first dose shouldnt require too much persuasion to get double jabbed.
# Would have been interesting to establish correlations between effect of vaccines and deaths as shown in Chart 3 above.
# However this isnt coming up in the matrix which does not factor time lags as it stands here.
# There is also a significant negative correlation between being fully Vaccinated and being hospitalised.
# This may imply that the vaccine reduces the severity of the infection.
# As such requirement to be hospitalised is reduced.
# However there could also be other factors at play which might explain this seemingly symbiotic relationship.
# Simple quantitative data analysis such as this wont reveal such factors.
# For example, it could be that in the early days, medical professionals under pressure panicked & over-hospitalised patients.
# Over time, as the panic subsided and doctors understood the infection better, they perhaps were more measured. 
# Thus hospitalisations dropped.
# We need qualitative data to investigate this further.

### Summaries & Observations from the Quantitative Data

There are some strong, consistent themes with respect to timing as the variables; the key highlights:

- There is a good amount of data albeit not real life credible; discussion on this aspect in the main report.
- As vaccines get rolled out, death declines
- The 'risky' variables appear to move in tandem with changes in vaccine uptake & numbers of fully vaccinated.
- The Province/State 'Others' should be stripped out for further study away from this exercise.
- The data is so vastly different in order of magnitude, its creating skews which are masking potential meaningful insights.
- The difference in magnitude makes meaningful comparative studies futile.
- One way to resolve this may be to aggregate all other various regions and then set up a comparative study with 'Others'.
- This may mask other important nuances of certain regions.
- Another method to resolve this problem of scale could be to normalise the whole data set.
- On normalisation, all variable values would fall within a predetermined range of values.
- Converting the data from daily to months made studying the data as time-series much easier.
- Daily observations are too noisy.
- Deaths have peaked in the summer of 2021 although they appear to be rising again.
- Both Deaths and Recovereds show very similar bi-modal distribution characteristics.
- Indicates there were two peaks over the summer of 2021.
- No idiosyncratic regional trends discovered yet.
- St. Helena, Ascension & Tristan da Cunha may be marginally better at recoveries although it wasnt so at the start of the pandemic.
- Most visualisations need a lot more work in terms of quality of presentation.
- The correlation matrix is a neat tool but assumes some basic statistical knowledge & may not be suitable for all audiences.
- There are many problems with this data set in terms of real life credibility; as such correlations should be treated with caution. They are intended to give a sense of trends and themes to the audience & not for use in decision making at this stage.
- Visualisations alone wont be sufficient to make decisions but should assist in generating the big picture storyboard in the minds of the government.
- There should be multivariate, spatial amongst other statistical analysis done on the data to generate actionable insights once the real world credibility of the underlying data is resolved.

## Analysing Twitter Data

#### Further look at the text column

In [ ]:
# List of values containing the # symbol.
tweets.loc[tweets['text'] != '#']

# Every single row contains at least one # symbol.

### Some basic descriptive analysis of the dataset tweets.csv


#### Averages

In [ ]:
# Average Number of times a tweet is retweeted.
tweets = tweets.sort_values(by='retweet_count', ascending=False)
tweets = tweets.reset_index(drop=True)

# Displaying results to nearest whole number
round(tweets['retweet_count'].mean(),0)

# We see than on average, a tweet is re-tweeted a further 3 times

In [ ]:
# Counting how many times a tweet has been re-tweeted.
# We have to revisit the original dataframe loaded from csv file
# Recall re_tweet count was dropped earlier in this exercise 
rt=tweets['retweet_count'] > 0
rt.value_counts()

# Shows that 1142 tweets were retweeted at least once.

In [ ]:
# Average number of likes 
tweets = tweets.sort_values(by='favorite_count', ascending=False)
tweets = tweets.reset_index(drop=True)

# Displaying results to nearest 2 decimals.
round(tweets['favorite_count'].mean(),2)

# We can see that an average tweet is marked favourite 10 times.

In [ ]:
# Count the number of times a tweet has been made favourite.
fav=tweets['favorite_count'] > 0
fav.value_counts()

# Shows that 1720 tweets were marked as favourite by someone at least once.

#### Top trending

In [ ]:
# Identify top trending hashtags.
# Ranked & limited to top 30 hashtags.
# These are the most frequently occuring string of texts that are either preceeded or followed by a '#'.
# To search for words starting with # we use (#.*?).
# To search for words or sentences ending #  (?=\s|$).

tweets2.text.str.findall(r'#.*?(?=\s|$)').head(31)

# This output isnt very useful. 
# Very quick scan of output does show that Covid 19 & coronavirus were very popular.
# # But cant make sense of how popular from the output in this format.

In [ ]:
# Creating a dataframe that contains just the text column from the tweets df.
# Converting the datatype in column 'text' as string.
tweets['text'] = tweets['text'].astype(str)

# Creating the dataframe.
tweets_text = tweets['text'].apply(lambda x: x if x.strip() != None else none)

# From the new df, creating a dict/list of all values with '#' in it.
tags = [x for x in tweets_text if x.startswith('#')]
for y in [x.split(' ') for x in tweets_text.values]:
    for z in y:
        if '#' in z:
            tags.append(z)


            
#Create a Series from the dictionary
tags=pd.Series(tags).value_counts()

In [ ]:
#Identify 30 most popular hashtags.
tags.head(30)

In [ ]:
# Convert into a dataframe
df = pd.DataFrame(tags).reset_index()
print(df.dtypes)

In [ ]:
# Rename columns
df.columns = ['word', 'count']
df['count'] = df['count'].astype(int)
df

In [ ]:
# Display hashtags which have been tweeted at least 100 times.
# This will further filter for significance of these hashtags.
df[df['count']>100]

# Can see actually very few really popular hashtags.
# Top 5 are all Covid-19 related.
# We should actually look for distinct words as a lot of the count is distorted by syntax.
# No real difference between COVID19, Covid19 & covid19 for example.
# Only distinct hashtags are Covid19, CovidIsNotOver, China and Greece.
# What was happening in Greece around this time?!

In [ ]:
# Visualising the table above.
g = sns.barplot(x="count", y="word", data=df.loc[(df['count']>100)])
g.set_title('Top Trending Hashtags')

# Very few meaningful insights here.
# Dataset is too small and over a very short period of time.
# Could see how these top trending hashtags evolve over the whole period of our quantitative analysis earlier.

### Summaries & Observations from Qualitative Data

The dataset provided is too small to provide meaningful insights.  Despite this, there are a handful strong, consistent themes that are worthwhile pointing out:

- Assuming that the data provided was from a period that was randomly selected, Twitter should be medium that the government targets to get its messaging on COVID across. 
- There is a significant level of animation amongst Twitter users in relation to COVID-19.
- Within the data provided, COVID was the highest trending hashtag by a great magnitude.
- Any government communication on the topic on Twitter will get a lot of eyeballs & can be thus very effective.
- Additional sentiment analysis would be a very worthwhile endeavour on a bigger dataset. 


### Week 6 Assignment
The government employed an external consultant to assist with data analysis. However, the consultant resigned and left the project in a half-completed state. The government provided you with the partially completed Jupyter Notebook that the consultant developed. They asked you to evaluate and complete the Jupyter Notebook with Python code.

This assignment activity will consist of two parts: (1) demonstrate the use of the functions provided, and (2) answer additional questions posed by the government. The questions are:

- What is the difference between qualitative and quantitative data? How can these be used in business predictions?
- Why is continuous improvement required? Can we not just implement the project and move on to other pressing matters?
- As a government, we adhere to all data protection requirements and have good governance in place. We only work with aggregated data and therefore will not expose any personal details. Does that mean we can ignore data ethics?


In [ ]:
# Create a special dataframe for the Province/State = Channel Islands
ds1 = pd.read_csv('covid_19_uk_cases.csv')
ds2 = pd.read_csv('covid_19_uk_vaccinated.csv')

sample = ds1[['Province/State','Date','Hospitalised']]

In [ ]:
# Select data for a specific province
sample_ci = sample[sample['Province/State'] == "Channel Islands"]

In [ ]:
# Define function to plot moving averages
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):
    
    # Create a rolling window to calculate the rolling mean using the series.rolling function
    rolling_mean = series.rolling(window=window).mean()
    
    # Declare the dimensions for the plot, plot name and plot the data consisting of the rolling mean from above 
    plt.figure(figsize=(18,4))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Simple moving average trend')

    
    # Plot confidence intervals for smoothed values
    if plot_intervals:
        
        # Calculate the mean absolute square 
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        
        # Calculate the standard deviation using numpy's std function
        deviation = np.std(series[window:] - rolling_mean[window:])
        
        # Calculate the upper and lower bounds 
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        
        # Name and style upper and lower bound lines and labels 
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
    
    # Plot the actual values for the entire timeframe
    plt.plot(series[window:], label='Actual values')
    plt.grid(True)

In [ ]:
# Define function to calculate the mean absolute error
def mean_absolute_error(a, b): return abs(b - a)

#### Question 1:

The consultant indicated that the function is functional, but did not demonstrate how to use it. Use the provided function, `plot_moving_average()`, to plot the data for the selected province (variable name is `sample_ci` and set the window parameter to 7 days. 

In [ ]:
# Demonstrate the use of the function to plot moving averages.

# First we need to ensure that we have all the environment set up.
# I have added a couple of libraries I need to the start cell.
# Converting the date column into datetime object.
# This is called parsing. 
# This is needed to calculate moving averages.
sample_ci['Date'] = pd.to_datetime(sample_ci['Date'], format="%d/%m/%Y")

# Calculating the 7 day moving average of Hospitalised numbers
# Inserting this calculated value as a new column within the existing dataframe, sample_ci
sample_ci['7d_MovingAvg'] = ((sample_ci['Hospitalised']).rolling(window=7).mean())

# We can change the number in the function above 'window=7' any other number that we wish to have to calculate moving average.
# For example if we wanted 20 day moving average, it would read window=5 in the function above.

# Now plotting this into a graph to show the 7 day moving average of the number of Hospitalisations in the Channel Islands.
# X Axis is time = 'Date'
# Y Axis = 7 day moving average of 'Hospitalised'

# Setting size of the plot.
plt.figure(figsize = (20,8))

# Visualising the graph.
g=sns.lineplot(x="Date", y="7d_MovingAvg", data=sample_ci)

# Set out chart title.
g.set_title('7day moving average of number of Hospitalisations in the Channel Islands')

# Moving averages are useful to identify the trend direction of a variable. 

### Question 2:

The consultant performed a calculation that looks interesting, but the team is unsure about what the intention was and how to interpret the output. Can you offer some insights into the meaning of the code and output in the cell below? Is it useful?

In [ ]:
# Return the top three days with biggest difference between daily value and rolling 7-day mean
s = sample_ci.copy()
s_rolling = s['Hospitalised'].rolling(window=7).mean()
s['error'] = mean_absolute_error(s['Hospitalised'][7:], s_rolling[7:])
s.sort_values('error', ascending=False).head(3)

- The above function calculates the 7day moving average of Hospitalisations in the Channel Islands.
- It also displays the actual number of Hospitalisations on the day. 
- Together, the two numbers are useful to identify the trend in which the actual number is going to move in the future. 
- If the moving average is less than the actual number on the day, it would typically signal that the actual numbers are on an uptrend and thus likely to continue to increase. There are other factors which may be used to better understand the signal provided by this difference between the moving average and the actual number on the day. However, this is out of scope for this assignment.
- The mean absolute error as shown in the Column 'error' in the table above shows the accuracy/strength of the signalling generated by the moving average as described above.
- The output in the table has been sorted in descending order of mean absolute error.
- Displayed in this manner, we can very easily identify the days when the predictive power of the 7d Moving Average was at its least capable. 
- Beyond this, displaying it in this manner has no value. Typically, given such studies are carried out on very large datasets, you would not display the output in this manner except to quickly check the worst case predictive power of the forecasting tool. 
- Output of this variety is very useful for predictive technologies especially for time series forecasting. It has little value in static data analysis.

### Question 3:
The management team had some additional questions around the project where they asked for further feedback to be included in your final presentation. Make sure to answer the questions in the Notebook in Markdown format in preparation for your presentation. The expectation is that you will provide short and direct responses to help them understand the importance and impact of the questions below.

#### What is the difference between qualitative and quantitative data? How can these be used in business predictions?

Quantitative data is numbers based and therefore measured. Qualitative data is descriptive and whilst it can be observed, there is no way to measure them. The techniques used for the collection and analysis is unique to each data type although certain subsets are shared.

Both data types have their place in being used for business predictions. Quantitative data is typically more readily available and thus useful for analysis on the fly. However, it can lack depth and on a stand-alone basis provides no context. Put simply, whilst quantitative data is useful for facts, qualitative data is a powerful story telling tool whilst keeping that story anchored in the real world. 

If you are running a business, it is crucial to know not just how your business is performing financially but also within its wider eco system with respect to customer and employee engagements and environmental impact amongst other measures.

For further discussion on this topic, I have found [this article](https://www.linkedin.com/pulse/20140920165433-34529931-qualitative-quantitative-decision-making/) to be particularly helpful. 

#### Can you provide you observations around why continuous improvement is required, can we not just implement the project and move on to other pressing matters?

Continuous improvement techniques are used as a strategy to proactively make consistent and iterative enhancements. These techniques can be used to improve products, processes and people. Simply put, it's the best way to ensure we are always at the boundaries of efficiency and knowledge . If continuous improvement processes are not in place and the focus instead is on task deliveries, over time productivity and market value will decline. Afterall, Rome wasn't built in a day but it did burn to the ground almost overnight!

#### As a government, we adhere to all data protection requirements and have good governance in place. Does that mean we can ignore data ethics? We only work with aggregated data and therefore will not expose any personal details? (Provide an example of how data ethics could apply to this case; two or three sentences max)

Data ethics whilst nuanced and complicated, cannot be ignored by any organisation that is not beyond the reach of the law and/or public opinion. Data can be supremely powerful but with great power comes great responsibility; primarily to ensure that the data is used for the appropriate use cases. 

In this specific case, using public health data which is ultimately aggregated from personal data can:

- cause further damage to public health and the economy if policy is formed basis erroneous data techniques.

- lead to breaches in confidentiality guaranteed by the law if the data is not handled appropriately. Ultimately, this could lead to legal action being taken against the government.